In [94]:
from collections import deque

from solution import *

In [95]:
file = 'calibration.txt'
cube_coords, max_zxy = parse_input(file)
new_cube_coords, grid = falling(cube_coords, max_zxy)
supports, is_supported_by = get_supports(new_cube_coords, grid)

In [113]:
def count_chain_reaction(cube_id, supports, is_supported_by):
    fallen = {cube_id}
    queue = deque(supports[cube_id])
    while queue:
        nxt = queue.popleft()
        if is_supported_by[nxt].issubset(fallen):
            fallen.add(nxt)
            queue.extend(supports[nxt])

    return max(0, len(fallen) - 1)

In [114]:
def main2(file):
    cube_coords, max_zxy = parse_input(file)
    new_cube_coords, grid = falling(cube_coords, max_zxy)
    supports, is_supported_by = get_supports(new_cube_coords, grid)

    sum_chain_reaction = 0
    for cube_id in supports:
        sum_chain_reaction += count_chain_reaction(cube_id, supports, is_supported_by)
    return sum_chain_reaction

In [115]:
main2('puzzle.txt')

75784